In [ ]:
# These values from Verda differ from the ones of the LRZ benchmark. Not entirely sure why.
# Verda
tpot_1_7b = 2.25
tpot_32b = 23.62

What kind of speedups should be obvserve from SD?
For $K=4$, we execute the drafter 4 times, and the target 1 time.
And because of token rejections, we get a number of token equivalent to the acceptance length (AL)

$$T_{s} = \frac{K \cdot T_{d} + T_{t}}{AL} = \frac{ITL}{AL} $$

In [ ]:
k = 4
itl = k * tpot_1_7b + tpot_32b
al = 3.67
sd_tpot = itl / al
sd_tpot

However, we are observing higher (worse, slower) TPOT. How slow is then the drafter really running?


$$T_{d} = \frac{T_{s} \cdot AL - T_{t}}{K}$$

In [ ]:
actual_sd_tpot = 11.43
implied_tpot_1_7b = (actual_sd_tpot * al - tpot_32b ) / k
implied_tpot_1_7b

In [ ]:
actual_itl = 4 * implied_tpot_1_7b + tpot_32b
actual_itl

However, this analysis must be with more tokens, not only 10.
How does batch size affect this? It changes AL for sure